In [1]:
import torch
import torch_geometric

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [2]:
import matplotlib as plt
import numpy as np
import networkx
from torch_geometric.data import Data
from sklearn.manifold import TSNE

In [3]:
from torch_geometric.utils import to_dense_adj

## Data Preparation on MUTAG

In [4]:
dataset = TUDataset(root="../dataset", name='MUTAG')

In [5]:
adj_o = to_dense_adj(dataset[2].edge_index)
adj_c = abs(to_dense_adj(dataset[2].edge_index) - 1) - torch.eye(len(dataset[2].x))

print("Original:\n", adj_o)
print("Complementary:\n", (adj_c))

Original:
 tensor([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]])
Complementary:
 tensor([[[0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 0., 0., 0., 

In [6]:
def toComplementary(g):
    c = abs(to_dense_adj(g.edge_index) - 1) - torch.eye(len(g.x))
    c = c[0].nonzero().t().contiguous()
    return c

In [7]:
dataset_c = []
for graph in dataset:
    edge_c = toComplementary(graph)
    dataset_c.append(Data(edge_index=edge_c, x=graph.x, y=graph.y))

In [8]:
dataset.y

tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0])

In [9]:
ys = []
for d in dataset_c:
    ys.append(d.y.item())
print(ys)

[1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]


In [10]:
# Train test split
ratio = 0.5
total = len(dataset)

# original graph
g_train = dataset[:round(ratio*total)]
g_test = dataset[round(ratio*total):]

# complementary graph
gc_train = dataset_c[:round(ratio*total)]
gc_test = dataset_c[round(ratio*total):]

In [11]:
print(f'g_train {g_train}')
print(f'g_test {g_test}')
print(f'gc_train {len(gc_train)}')
print(f'gc_test {len(gc_test)}')

g_train MUTAG(94)
g_test MUTAG(94)
gc_train 94
gc_test 94


In [12]:
print([x.y.item() for x in g_train])

[1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1]


In [13]:
print([x.y.item() for x in gc_train])

[1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1]


In [14]:
bs = 32
seed = 12345

g_train_loader = DataLoader(g_train, batch_size=bs, shuffle=False)
g_test_loader = DataLoader(g_test, batch_size=bs, shuffle=False)

gc_train_loader = DataLoader(gc_train, batch_size=bs, shuffle=False)
gc_test_loader = DataLoader(gc_test, batch_size=bs, shuffle=False)

In [15]:
for g in g_train_loader:
    print(g.y)
#     break

tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1])
tensor([0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 1])


In [16]:
for g in g_test_loader:
    print(g.y)
    break

tensor([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 1, 0, 0, 1, 1])


In [17]:
for g in gc_train_loader:
    print(g.y)
#     break

tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1])
tensor([0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 1])


In [18]:
for g in gc_test_loader:
    print(g.y)
    break

tensor([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 1, 0, 0, 1, 1])


## Building model

In [19]:
from torch_geometric.nn import GCNConv
from torch.nn import Linear
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_max_pool
from torch_geometric.nn import global_add_pool
import torch.nn.functional as F

In [299]:
class ComplementarySupCon(torch.nn.Module):
    def __init__(self, dataset, hidden_channels):
        super(ComplementarySupCon, self).__init__()
        
        # weight seed
        torch.manual_seed(42)
        self.conv1_o = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2_o = GCNConv(hidden_channels, hidden_channels)
        self.conv3_o = GCNConv(hidden_channels, hidden_channels)
        
        self.conv1_c = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2_c = GCNConv(hidden_channels, hidden_channels)
        self.conv3_c = GCNConv(hidden_channels, hidden_channels)

    def forward(self, x_o, x_c, edge_index_o, edge_index_c, batch_o):
        x_o = self.conv1_o(x_o, edge_index_o)
        x_o = x_o.relu()
        x_o = self.conv2_o(x_o, edge_index_o)
        x_o = x_o.relu()
        x_o = self.conv3_o(x_o, edge_index_o)

        x_c = self.conv1_c(x_c, edge_index_c)
        x_c = x_c.relu()
        x_c = self.conv2_c(x_c, edge_index_c)
        x_c = x_c.relu()
        x_c = self.conv2_c(x_c, edge_index_c)


        h = x_o + x_c
        
        h = global_add_pool(h, batch_o)
        
        return h, x_o, x_c

In [209]:
model = ComplementarySupCon(dataset, 64)

In [178]:
model

ComplementarySupCon(
  (conv1_o): GCNConv(7, 64)
  (conv2_o): GCNConv(64, 64)
  (conv1_c): GCNConv(7, 64)
  (conv2_c): GCNConv(64, 64)
)

In [203]:
# Based on Khosla 2020 - Supervised contrastive learning

def supervisedContrastiveLoss(embeddings, labels, tau):
    loss = 0
    outer = 0
    inner = 0
    denom = 0

    # outer
    # z_i = embeddings of graph i
    # z_i = label of graph i
    for out_index, (z_i, y_i) in enumerate(zip(embeddings, labels)):
        Pi = torch.sum(labels == y_i)
        # loss = z_i
        # loop to all positive pair with z_i and skip z_i
        for in_index, (zp_i, lp_i) in enumerate(zip(embeddings, labels)):
            if lp_i != y_i or out_index == in_index:
                continue
            num = torch.exp(torch.matmul(z_i, zp_i)/tau)
            # calculate denumerator
            for _, za_i in enumerate(embeddings):
                # only take zi != za_i
                if out_index == in_index:
                    continue
                denom = denom + torch.exp(torch.matmul(z_i, za_i)/tau)
                
            inner = inner + torch.log(num/denom)
        
        outer = outer + (-1 / Pi * inner)

        # reset inner denom and inner
        denom, inner = 0, 0
        
    
    loss = outer            
    # print(loss)
    return outer, embeddings

In [198]:
h, x_o, x_c = None, None, None
loss = None
y = None
for index, (g_o, g_c) in enumerate(zip(g_train_loader, gc_train_loader)):
    h, x_o, x_c = (model(g_o.x, g_c.x, g_o.edge_index, g_c.edge_index, g_o.batch))
    loss, y = supervisedContrastiveLoss(h, g_o.y, 100)
    # break

tensor(182.2872, grad_fn=<AddBackward0>)
tensor(179.7015, grad_fn=<AddBackward0>)
tensor(164.2228, grad_fn=<AddBackward0>)


In [197]:
# torch.autograd.set_detect_anomaly(False)

In [300]:
def train(model, g_loader, gc_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    model.train()
    
    for _, (g_o, g_c) in enumerate(zip(g_loader, gc_loader)):
        h, x_o, x_c = (model(g_o.x, g_c.x, g_o.edge_index, g_c.edge_index, g_o.batch))
        loss, _ = supervisedContrastiveLoss(h, g_o.y, 32)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return h, loss

@torch.no_grad()
def test(model, loader):
    model.eval()
    for data in loader:
        z, loss = model(data.x, data.edge_index, data.batch)
    return z, loss

In [301]:
model = ComplementarySupCon(dataset, 64)

list_loss = []
list_train_acc = []
list_test_acc = []
z = None
# torch.autograd.set_detect_anomaly(True)
for epoch in range(0, 50):
    h, loss = train(model, g_train_loader, gc_train_loader)
    print(f"epoch: {epoch+1} training loss: {loss:.4f}")
    # print(f"epoch: {epoch+1} training loss: {loss:.4f}; test loss: {loss_test:.4}")
    # break


epoch: 1 training loss: 161.6641
epoch: 2 training loss: 156.5426
epoch: 3 training loss: 147.9382
epoch: 4 training loss: 147.1410
epoch: 5 training loss: 147.1851
epoch: 6 training loss: 147.1852
epoch: 7 training loss: 147.1653
epoch: 8 training loss: 147.0699
epoch: 9 training loss: 147.1870
epoch: 10 training loss: 147.1501
epoch: 11 training loss: 147.1663
epoch: 12 training loss: 147.1658
epoch: 13 training loss: 147.1409
epoch: 14 training loss: 147.0844
epoch: 15 training loss: 147.1079
epoch: 16 training loss: 147.0854
epoch: 17 training loss: 147.0117
epoch: 18 training loss: 147.0333
epoch: 19 training loss: 147.0114
epoch: 20 training loss: 146.8990
epoch: 21 training loss: 146.7960
epoch: 22 training loss: 146.9786
epoch: 23 training loss: 146.6019
epoch: 24 training loss: 147.2462
epoch: 25 training loss: 147.0716
epoch: 26 training loss: 146.9159
epoch: 27 training loss: 146.9176
epoch: 28 training loss: 147.6386
epoch: 29 training loss: 146.6279
epoch: 30 training loss

In [290]:
embeddings = [] 
labels = [] 
color_list = ['red', 'blue']
color_list2 = ['white', 'gray']

for _, (g_o, g_c) in enumerate(zip(g_train_loader, gc_train_loader)):
    h, x_o, x_c = (model(g_o.x, g_c.x, g_o.edge_index, g_c.edge_index, g_o.batch))
    for emb in h:
        # print(emb.detach().numpy())|
        embeddings.append(emb.detach().numpy())
    labels += [color_list[y-1] for y in g_o.y]

for _, (g_o, g_c) in enumerate(zip(g_test_loader, gc_test_loader)):
    h, x_o, x_c = (model(g_o.x, g_c.x, g_o.edge_index, g_c.edge_index, g_o.batch))
    for emb in h:
        # print(emb.detach().numpy())|
        embeddings.append(emb.detach().numpy())
    labels += [color_list[y-1] for y in g_o.y]

In [291]:
embeddings = np.array(embeddings)
embeddings

array([[-0.36881438, -0.12340486,  0.02199826, ...,  0.0451711 ,
         0.13098003,  0.35874978],
       [-0.7055712 ,  0.15735087,  0.3552161 , ..., -0.11775713,
         0.11129105,  0.4992735 ],
       [-0.70408946,  0.15702295,  0.35772353, ..., -0.12094232,
         0.11112064,  0.5003334 ],
       ...,
       [-0.6827925 ,  0.16901954,  0.37004864, ..., -0.13747919,
         0.11617702,  0.47918075],
       [-0.270846  , -0.2744132 , -0.07618007, ...,  0.04173164,
         0.20364578,  0.41225892],
       [-0.65976477,  0.05680251,  0.29274914, ..., -0.09539726,
         0.12269305,  0.5188308 ]], dtype=float32)

In [292]:
tsne = TSNE(n_components=3, random_state=42)
X_tsne = tsne.fit_transform(embeddings)
# tsne.kl_divergence_

In [293]:
import plotly.express as px

In [302]:
fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], color=labels)
fig.update_layout(
    title="GCN + SupCon + Complementary Graph",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()